Project: **Machine Learning Programming Project 2 Part 2** 
<br>
Team Members: **Debit Paudel, Kushal Dahal**
<br>
We have used github for the collaboration.
<br>
Github Link: https://github.com/debit7/Bayes_Algorithm_ML


While looking at the dataset provided by the analytics department with the hits for every hour of the first month, we can see that on the last 5-6 days, the hits are larger as compared to the previous days. We think when one person read the blog and like it, he/she may refer to other friends. So looking at the data, we can say that "word of mouth" may have a significant role to increase the hits. Also, we can assume that more people are reading the blog as more longer it is active. Hence, we can expect more traffic in the future rather than this first month. After visualizing the growth of hits and predicting the future hits building a linear regression model, we will be statistically clear.

In [161]:
import pandas as pd
with open('spam.data') as f:
        lst = []
        for ele in f:
            line = ele.replace('\n','').split('\t')
            
            lst.append(line)
Headers=['Classifier','Messages']
df = pd.DataFrame(lst,columns =Headers) 
df.head()


,Classifier,Messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [162]:
#randomly splitting data in training and testing data set
training = df.sample(frac=0.75)
testing = df.drop(training.index)
#print(training)
#print(test)


In [163]:
#removing punctuations from the messages and converting the upper cases to lower case
punctuations='''€˜%^&"\,!*_~)(-[};:]{'<#£$>./?@+'''
stop_words=["i","da","we","ur" ,"u","am","me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
wordlist=[]
for message in training['Messages']:
    
    #print(message,'\n')
    for alpha in message:
        if alpha in punctuations:
            message = message.replace(alpha, " ")
    #for lower case        
    message=message.lower()
    
    #for stop words
    message=message.split()
    #print(message)
    for word in message:
        if word in stop_words:
            #print(word,'\n')
            message.remove(word)
        wordlist.append(word)
wordlist=list(set(wordlist))
Vocabulary = len(wordlist) 

    
    

In [164]:
#calculating frequency for each word in a message
word_counts_per_Messages=[
    [row[1].count(word) for word in wordlist]
    for _, row in training.iterrows()]
df_wordcount=pd.DataFrame(word_counts_per_Messages,columns=wordlist)
training = pd.concat([training.reset_index(), df_wordcount], axis=1).iloc[:,1:]


In [165]:
spam_probability = training['Classifier'].value_counts()['spam'] / len(training)
ham_probability = training['Classifier'].value_counts()['ham'] / len(training)
spam_list=training['Classifier'] == 'spam'
ham_list=training['Classifier'] == 'ham'
spam = training[training['Classifier'] == 'spam']
ham = training[training['Classifier'] == 'ham']
spam_n = training.loc[spam_list, 'Messages'].apply(len).sum()
ham_n = training.loc[ham_list, 'Messages'].apply(len).sum()


In [166]:
def probability_word_spam(word):
        return (spam[word].sum() + 1) / (spam_n + Vocabulary) if word in training.columns else 1
   
def probability_word_ham(word):
        return (ham[word].sum() + 1) / (ham_n + Vocabulary) if word in training.columns else 1
    

In [167]:
import math as mt
def naive_classifier(Message):
    prob_spam = spam_probability
    prob_ham = ham_probability
    for word in Message:
        prob_spam += mt.log(probability_word_spam(word),10)
        prob_ham += mt.log(probability_word_ham(word),10)
    prob_spam=10**prob_spam
    prob_ham=10**prob_ham
    return 'ham' if prob_ham > prob_spam else 'spam'
    

In [168]:
testing['Predicted_label'] = testing['Messages'].apply(naive_classifier)
Accuracy = (testing['Predicted_label'] == testing['Classifier']).sum() / len(test) * 100
print("Accuracy of this model:",Accuracy)

Accuracy of this model: 96.12625538020086
